# Imports

In [1]:
import logging
import shutil
import sys
from os.path import isdir, join
from tempfile import gettempdir
from uuid import uuid4

import folium
import pandas as pd
from aequilibrae import Parameters, Project, logger

from functions.basic_stats import basic_stats
from functions.bridge_stats import bridge_stats
from functions.toll_stats import toll_stats
from functions.tunnel_stats import tunnel_stats

# Model place (Country) to build the model for

In [2]:
model_place = 'Andorra'

In [3]:
fldr = f'/home/jovyan/workspace/road_analytics/{model_place}'

if isdir(fldr):
    print(fldr)

/home/jovyan/workspace/road_analytics/Andorra


# Open Model

In [4]:
project = Project()
project.open(fldr)

# Get model data

In [5]:
links = pd.read_sql('select * from links', project.conn).drop(columns=['geometry'])

# Print statistics

In [6]:
basic_stats(links)

----- Links -----

|   General |   Bridge |   Toll |   Tunnel |   Total |
|-----------|----------|--------|----------|---------|
|      4641 |      288 |     15 |       63 |    5007 |

----- Total km -----

|   General |   Bridge |   Toll |   Tunnel |   Total |
|-----------|----------|--------|----------|---------|
|    478.20 |     7.54 |   4.49 |    16.55 |  506.77 |

----- Link Type -----

| Link Type     |   Links |     km |
|---------------|---------|--------|
| living_street |      54 |   2.02 |
| primary       |    1225 | 117.03 |
| residential   |    1449 | 114.44 |
| secondary     |     910 | 145.93 |
| service       |     856 |  57.68 |
| tertiary      |     296 |  42.80 |
| trunk         |      63 |  12.82 |
| unclassified  |     153 |  13.71 |
| via_ferrata   |       1 |   0.34 |

----- Pavement Surfaces -----

| Surface Type   |   Links |     km |
|----------------|---------|--------|
| asphalt        |    2760 | 341.95 |
| compacted      |       2 |   0.53 |
| concrete   

In [7]:
bridge_stats(links)

----- Bridge - Overall Stats -----
 
|        |   Links |        km |
|--------|---------|-----------|
| Bridge |     288 |   7536.45 |
| Total  |    5007 | 506772.07 |
 
There are no toll bridges in the search area.

----- Bridge - Link Type -----

| Link Type     |   Links |   km |
|---------------|---------|------|
| living_street |       1 | 0.01 |
| primary       |     106 | 3.42 |
| residential   |      57 | 0.89 |
| secondary     |      57 | 1.21 |
| service       |      29 | 0.54 |
| tertiary      |      11 | 0.11 |
| trunk         |      12 | 1.06 |
| unclassified  |      15 | 0.29 |

----- Bridge - Pavement Surface -----

| Surface Type   |   Links |   km |
|----------------|---------|------|
| asphalt        |     185 | 5.27 |
| unclassified   |     103 | 2.26 |



In [8]:
toll_stats(links)

----- Tolls - Overall Stats -----
 
|       |   Links |     km |
|-------|---------|--------|
| Tolls |      15 |   4.49 |
| Total |    5007 | 506.77 |
 
----- Tolls - Link Type -----

| Link Type   |   Links |   km |
|-------------|---------|------|
| primary     |       2 | 0.22 |
| trunk       |      13 | 4.27 |

----- Tolls - Pavement Surfaces -----

| Surface Type   |   Links |   km |
|----------------|---------|------|
| asphalt        |      15 | 4.49 |



In [9]:
tunnel_stats(links)

----- Tunnels - Overall Stats -----
 
|         |   Links |     km |
|---------|---------|--------|
| Tunnels |      63 |  16.55 |
| Total   |    5007 | 506.77 |
 
----- Toll Tunnels -----
 
|   links |   km |
|---------|------|
|       2 | 2.96 |

----- Tunnels - Link Type -----

| Link Type     |   Links |   km |
|---------------|---------|------|
| living_street |       4 | 0.03 |
| primary       |      16 | 3.65 |
| residential   |       4 | 0.14 |
| secondary     |       2 | 0.09 |
| service       |      19 | 3.40 |
| tertiary      |       4 | 0.37 |
| trunk         |       9 | 8.81 |
| unclassified  |       5 | 0.05 |

----- Tunnels - Pavement Surfaces -----

| Surface Type   |   Links |    km |
|----------------|---------|-------|
| asphalt        |      33 | 13.03 |
| unclassified   |      30 |  3.52 |



In [10]:
project.close()